In [1]:
import os
import pandas as pd

def run_command(cmd):
    import subprocess
    process = subprocess.Popen(cmd.split(), stdout=subprocess.PIPE)
    output, error = process.communicate()
    return output.decode('utf-8')

def predict_partition(self, temperature_k =None, dms = None):
    # predict the partition of rna structures
    cmd = f"{os.path.join(self.rnastructure_path, 'partition')} {self.fasta_file} {self.pfs_file}"
    if temperature_k != None:
        cmd += ' --temperature '+str(temperature_k)
    if type(dms) != type(None):
        assert len(self.sequence) == len(dms), 'The length of the sequence is not the same as the length of the signal.'
        assert type(dms) in [list, tuple, np.ndarray], 'The dms signal should be a list of floats.'
        self.__write_dms_to_file(self.sequence, dms)
        cmd += ' --dms ' + self.dms_file
    run_command(cmd)
    
    # sum it into pairing probability
    run_command(os.path.join(self.rnastructure_path,'ProbabilityPlot')+ ' ' +self.pfs_file + ' -t '+self.prob_file)
    
    # read the probability file 
    with open(self.prob_file,"r") as f:
        lines=f.readlines()
        pairingPrediction={'i':[],'j':[],'p':[]}
        for x in range(len(lines)):
            if x>1:
                ls=lines[x].split("\t")
                pairingPrediction["i"]+=[int(ls[0])]
                pairingPrediction["j"]+=[int(ls[1])]
                pairingPrediction["p"]+=[float(ls[2])]
    df = pd.DataFrame(pairingPrediction)